In [ ]:
import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import pyjokes
import requests, json, sys
import pygame
import pyaudio
import os
import sympy as sp

In [ ]:
#creating a speech recognition object
listener=sr.Recognizer()

listener.energy_threshold = 1000

#Initialize the pygame mixer for playing audio
pygame.mixer.init()

#initialise TTS engine
engine=pyttsx3.init()

voices=engine.getProperty("voices")
engine.setProperty('voice',voices[1].id)

In [ ]:
#Function to greet the user at the start of the program
def greet_user():
    engine_talk("Hey, I'm Alex, your virtual assistant. How can I help you today?")
    pygame.mixer.music.load("C:/Users/Arya Verma/Downloads/ding-36029.mp3")

In [ ]:
#Function to fetch weather data for a given city using the OpenWeatherMap API
def weather(city):
    api_key = "984e20e499894d79ec0f8c5727c425b3"
    base_url = "http://api.openweathermap.org/data/2.5/weather?" 
    city_name = city
    complete_url = base_url + "appid=" + api_key + "&q=" + city_name + "&units=metric"    
    #Send a GET request to the API and get the response in JSON format
    response = requests.get(complete_url) 
    x = response.json()     
    if x["cod"] != "404":
        #Extract the main weather data (including temperature) from the JSON response
        y = x["main"]
        current_temperature = y["temp"]
        return True, current_temperature
    else:
        return False, None

In [ ]:
#Function to convert text to speech and play the audio
def engine_talk(text):
    text = text.replace('-', 'minus ')
    text = text.replace('/', 'divided by ')
    text = text.replace('**', 'to the power ')
    engine.say(text)
    engine.runAndWait()

In [ ]:
#Function to listen for user commands through the microphone
def user_commands():
    command = ""
    try:
        with sr.Microphone() as source:
            print("Start Speaking!!")
            pygame.mixer.music.play() 
            listener.adjust_for_ambient_noise(source)  # Adjust for ambient noise
            audio = listener.listen(source, timeout=7)  # Set a timeout of 15 seconds
            command = listener.recognize_google(audio)
            command = command.lower()
            if 'alex' in command:
                command = command.replace('alex', '')              
    except:
        pass
    return command

In [ ]:
def integrate_expression(expression):
    x = sp.symbols('x')
    
    # Handle exponentiation
    while ' to the power ' in expression:
        expression = expression.replace(" to the power ", "^")        
    # Add parentheses around functions without parentheses
    expression = expression.replace(" x", "(x)")
    # Handle nesting
    while ' of ' in expression:
        expression = expression.replace(" of ", "(")
        
    open_parentheses = expression.count('(')
    close_parentheses = expression.count(')')
    
    # Add closing parentheses until the counts are balanced
    while open_parentheses > close_parentheses:
        expression += ")"
        close_parentheses += 1
    
    integrand = sp.sympify(expression)
    integral = sp.integrate(integrand, x)
    return integral

In [ ]:
def differentiate_expression(expression):
    x = sp.symbols('x')
    # Handle exponentiation
    while ' to the power ' in expression:
        expression = expression.replace(" to the power ", "^")        
    # Add parentheses around functions without parentheses
    expression = expression.replace(" x", "(x)")
    
    while ' of ' in expression:
        expression = expression.replace(" of ", "(")
    open_parentheses = expression.count('(')
    close_parentheses = expression.count(')')
    
    # Add closing parentheses until the counts are balanced
    while open_parentheses > close_parentheses:
        expression += ")"
        close_parentheses += 1
       
    expr = sp.sympify(expression)
    derivative = sp.diff(expr, x)
    return derivative

In [ ]:
#Function to process user commands and provide responses
def run_alex():
    command=user_commands()
    print("Command:", command)
    if 'play' in command:
        song = command.replace('play', '')
        engine_talk('Playing ' + song)
        pywhatkit.playonyt(song)
        os._exit(0)
    elif 'time' in command:
        time=datetime.datetime.now().strftime('%I:%M:%p')
        engine_talk('the current time is'+time)
        print(time)
    elif 'who is' in command:
        name=command.replace('who is','')
        try:
            info=wikipedia.summary(name,sentences=3)
            print(info)
            engine_talk(info)
        except wikipedia.exceptions.PageError as e:
            engine_talk('No results found. Please check your query.')
    elif 'joke' in command:
        joke=pyjokes.get_joke()
        engine_talk(joke)
        print(joke)
    elif 'weather' in command:
        engine_talk('Please tell me the name of your city')
        city = user_commands()
        while not city:
            engine_talk('Sorry, I did not catch the city name. Please repeat the city name.')
            city = user_commands()
        success, temperature = weather(city)
        if success:
            engine_talk('The current temperature in ' + city + ' is ' + str(temperature) + ' degrees Celsius.')
            print(str(temperature))
        else:
            engine_talk('Sorry, I could not find the weather data for ' + city)
    elif 'what is' in command:
        # Extract the expression from the command
        expression = command.replace('what is', '')

        # Check for addition
        if '+' in expression:
            numbers = expression.split('+')
            if len(numbers) == 2:
                try:
                    num1 = float(numbers[0])
                    num2 = float(numbers[1])
                    result = num1 + num2
                    engine_talk(f"The result of {expression} is {result}.")
                    print("result:",result)
                except ValueError:
                    engine_talk("Sorry, I couldn't evaluate the expression.")
            else:
                engine_talk("Sorry, I couldn't understand the expression.")

        # Check for subtraction
        elif '-' in expression:
            numbers = expression.split('-')
            if len(numbers) == 2:
                try:
                    num1 = float(numbers[0])
                    num2 = float(numbers[1])
                    result = num1 - num2
                    engine_talk(f"The result of {expression} is {result}.")
                    print("result:",result)
                except ValueError:
                    engine_talk("Sorry, I couldn't evaluate the expression.")
            else:
                engine_talk("Sorry, I couldn't understand the expression.")

        # Check for multiplication
        elif 'into' in expression:
            numbers = expression.split('into')
            if len(numbers) == 2:
                try:
                    num1 = float(numbers[0])
                    num2 = float(numbers[1])
                    result = num1 * num2
                    engine_talk(f"The result of {expression} is {result}.")
                    print("result:",result)
                except ValueError:
                    engine_talk("Sorry, I couldn't evaluate the expression.")
            else:
                engine_talk("Sorry, I couldn't understand the expression.")

        # Check for division
        elif 'by' in expression:
            numbers = expression.split('by')
            if len(numbers) == 2:
                try:
                    num1 = float(numbers[0])
                    num2 = float(numbers[1])
                    if num2 != 0:
                        result = num1 / num2
                        engine_talk(f"The result of {expression} is {result}.")
                        print("result:",result)
                    else:
                        engine_talk("Sorry, division by zero is not allowed.")
                except ValueError:
                    engine_talk("Sorry, I couldn't evaluate the expression.")
            else:
                engine_talk("Sorry, I couldn't understand the expression.")

        else:
            engine_talk("Sorry, I couldn't understand the expression.")

    elif 'add' in command:
        # Extract the numbers from the command
        numbers = command.replace('add', '').split('and')

        if len(numbers) == 2:
            try:
                num1 = float(numbers[0])
                num2 = float(numbers[1])

                # Perform addition
                result = num1 + num2
                engine_talk(f"The result of adding {num1} and {num2} is {result}.")
                print("result:",result)
            except ValueError:
                engine_talk("Sorry, I couldn't perform the addition.")
        else:
            engine_talk("Sorry, I couldn't understand the command.")

    elif 'subtract' in command:
        # Extract the numbers from the command
        numbers = command.replace('subtract', '').split('from')

        if len(numbers) == 2:
            try:
                num1 = float(numbers[1])
                num2 = float(numbers[0])

                # Perform subtraction
                result = num1 - num2
                engine_talk(f"The result of subtracting {num2} from {num1} is {result}.")
                print("result:",result)
            except ValueError:
                engine_talk("Sorry, I couldn't perform the subtraction.")
        else:
            engine_talk("Sorry, I couldn't understand the command.")

    elif 'multiply' in command:
        # Extract the numbers from the command
        numbers = command.replace('multiply', '').split('and')

        if len(numbers) == 2:
            try:
                num1 = float(numbers[0])
                num2 = float(numbers[1])

                # Perform multiplication
                result = num1 * num2
                engine_talk(f"The result of multiplying {num1} and {num2} is {result}.")
                print("result:",result)
            except ValueError:
                engine_talk("Sorry, I couldn't perform the multiplication.")
        else:
            engine_talk("Sorry, I couldn't understand the command.")

    elif 'divide' in command:
        # Extract the numbers from the command
        numbers = command.replace('divide', '').split('by')

        if len(numbers) == 2:
            try:
                num1 = float(numbers[0])
                num2 = float(numbers[1])

                if num2 != 0:
                    # Perform division
                    result = num1 / num2
                    engine_talk(f"The result of dividing {num1} by {num2} is {result}.")
                    print("result:",result)
                else:
                    engine_talk("Sorry, division by zero is not allowed.")
            except ValueError:
                engine_talk("Sorry, I couldn't perform the division.")
        else:
            engine_talk("Sorry, I couldn't understand the command.")
    elif 'integrate' in command:
        # Extract the expression from the command
        expression = command.replace('integrate', '').strip()
        #print(expression)
        try:
            result = integrate_expression(expression)
            engine_talk(f"The integral of {expression} is {result}.")
            print("result:")
            print(result)
        except (sp.SympifyError, ValueError):
            engine_talk("Sorry, I couldn't perform the integration.")
       
    elif 'differentiate' in command:
        # Extract the expression from the command
        expression = command.replace('differentiate', '').strip()
        #print(expression)
        try:
            result = differentiate_expression(expression)
            engine_talk(f"The derivative of {expression} is {result}.")
            print("result:")
            print(result)
        except (sp.SympifyError, ValueError):
            engine_talk("Sorry, I couldn't perform the differentiation.")
        
    elif 'exit' in command:
        engine_talk('Signing off!!')
        os._exit(0) 
    else:
        engine_talk('Sorry, I could not understand your request.')


In [ ]:
greet_user()
#Start the main loop to continuously listen for user commands
while True:
    run_alex()